In [1]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import time
import numpy as np
from netCDF4 import Dataset
from matplotlib import animation, rc
from matplotlib import pyplot as plt
import numpy as np

#Finally, import our simulator
from SWESimulators import CTCS, KP07, DataOutput

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

In [2]:
#Set up simulation parameters for case 1A (uniform along shore wind stress)
nx = 50
ny = 25

dx = 20000
dy = 20000

nt =2500
dt = 90.0
g = 9.81
f = 0.00012
r = 0.0024
A = 0.001
wind_type = 1
wind_tau0 = 0.1
wind_rho = 1025.0
wind_alpha = 0.000005

wind_xm = dx*(0.5 + nx/2)

wind_Rc = 200000.0
wind_u0 = 30.0/np.sqrt(5.0)
wind_v0 = -0.5*wind_u0
wind_x0 = dx*(nx/2) - wind_u0*3600.0*48.0
wind_y0 = dy*(ny/2) - wind_v0*3600.0*48.0

h0 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 50.0;
eta0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
u0 = np.zeros((ny+2, nx+1), dtype=np.float32, order='C');
v0 = np.zeros((ny+1, nx+2), dtype=np.float32, order='C');


#Initialize simulator
reload(CTCS)
sim = CTCS.CTCS(h0, eta0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, A, \
                wind_type, \
                wind_tau0, wind_rho, wind_alpha)




#Calculate radius from center of bump for plotting
fig = plt.figure()

ax = fig.add_subplot(1, 3, 1)
p0 = plt.imshow(eta0[1:-1, 1:-1].T, interpolation='spline36', origin='bottom')
plt.colorbar()

ax = fig.add_subplot(1, 3, 2)
p1 = plt.imshow(u0[1:-1, :].T, interpolation='spline36', origin='bottom')
plt.colorbar()

ax = fig.add_subplot(1, 3, 3)
p2 = plt.imshow(v0[:, 1:-1].T, interpolation='spline36', origin='bottom')
plt.colorbar()


def animate(i):
    if (i>0):
        t = sim.step(3600.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download()

    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)
    
    fig.add_subplot(1, 3, 1)
    p0.set_data(eta1[1:-1, 1:-1].T)
    
    fig.add_subplot(1, 3, 2)
    p1.set_data(u1[1:-1, :].T)
    
    fig.add_subplot(1, 3, 3)
    p2.set_data(v1[:, 1:-1].T)
    
    plt.draw()

anim = animation.FuncAnimation(fig, animate, range(96), interval=100)
plt.close(anim._fig)
anim

Using  GeForce 840M


In [3]:
#Set up simulation parameters for case 1A (uniform along shore wind stress)
nx = 50
ny = 25

dx = 20000
dy = 20000

nt =2500
dt = 90.0
g = 9.81
f = 0.00012
r = 0.0024
A = 0.001
theta = 1.3
use_rk2 = True
wind_type = 1
wind_tau0 = 0.1
wind_rho = 1025.0
wind_alpha = 0.000005

wind_xm = dx*(0.5 + nx/2)

wind_Rc = 200000.0
wind_u0 = 30.0/np.sqrt(5.0)
wind_v0 = -0.5*wind_u0
wind_x0 = dx*(nx/2) - wind_u0*3600.0*48.0
wind_y0 = dy*(ny/2) - wind_v0*3600.0*48.0

h0 = np.ones((ny+4, nx+4), dtype=np.float32, order='C') * 50.0;
u0 = np.zeros((ny+4, nx+4), dtype=np.float32, order='C');
v0 = np.zeros((ny+4, nx+4), dtype=np.float32, order='C');


#Initialize simulator
reload(KP07)
sim = KP07.KP07(h0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                theta, use_rk2, \
                wind_type, \
                wind_tau0, wind_rho, wind_alpha)




#Calculate radius from center of bump for plotting
fig = plt.figure()

ax = fig.add_subplot(1, 3, 1)
p0 = plt.imshow(h0[2:-2, 2:-2].T - 50.0, interpolation='nearest', origin='bottom')
plt.colorbar()

ax = fig.add_subplot(1, 3, 2)
p1 = plt.imshow(u0[2:-2, 2:-2].T, interpolation='nearest', origin='bottom')
plt.colorbar()

ax = fig.add_subplot(1, 3, 3)
p2 = plt.imshow(v0[2:-2, 2:-2].T, interpolation='nearest', origin='bottom')
plt.colorbar()


def animate(i):
    if (i>0):
        t = sim.step(3600.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()

    fig.suptitle("KP07 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)
    
    fig.add_subplot(1, 3, 1)
    p0.set_data(h1[2:-2, 2:-2].T - 50.0)
    
    fig.add_subplot(1, 3, 2)
    p1.set_data(u1[2:-2, 2:-2].T)
    
    fig.add_subplot(1, 3, 3)
    p2.set_data(v1[2:-2, 2:-2].T)
    
    plt.draw()   

    fig.savefig("{:010.0f}_kp07.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(96), interval=100)
plt.close(anim._fig)
anim

Using  GeForce 840M
